## Load python libraries

In [29]:
import configparser
import gzip
import os
import glob
import subprocess
import multiprocessing
import shutil
import pandas as pd

## Import information from configuration file

In [30]:
config = configparser.ConfigParser()
config.read('config.ini')

path_data = config.get('paths', 'path_data')

print(path_data)

../data


## Define functions

In [51]:
'''
Function to obtain paths to the directories where the fastq files are located.
Currently requires demultiplexed data with gzipped files.
'''
def fastq_directory_paths():
    fastq_dir = set()
    file_paths = glob.glob(path_data + '/**/*.fastq.gz', recursive=True)
    directory_paths = map(os.path.dirname, file_paths)
    fastq_dir.update(directory_paths)
    return sorted(fastq_dir)

'''
Function to obtain sample names that are used as base to track the samples through the analysis
'''
basename = lambda paths: [path.split('/')[-2] if path.endswith('/') else path.split('/')[-1] for path in paths]

'''
Functions to count the number of raw sequences per sample
'''
def count_sequences(file_path):
    count = 0
    with gzip.open(file_path, 'rt') as gz_file:
        for line in gz_file:
            if line.startswith('@'):
                count += 1
    return count

def accumulate_counts(folder_path):
    counts = {}
    for folder in folder_path:
        counts[folder] = 0  # Initialize the folder key with count 0
        for file_name in os.listdir(folder):
            file_path = os.path.join(folder, file_name)
            if file_name.endswith('.fastq.gz'):
                counts[folder] += count_sequences(file_path)
    return counts

'''
Wrapper function for NanoFilt
'''
def filtering(folder_list, minimum_length, maximum_length, qscore, base_list):
    for folder, base in zip(folder_list, base_list):
        if os.path.exists(os.path.join('results/qc', base)) and os.path.isdir(os.path.join('results/qc', base)):
            pass
        else:
            os.makedirs(os.path.join('results/qc', base))
        for file_name in os.listdir(folder):
            if file_name.endswith('.fastq.gz'):
                file_path = os.path.join(folder, file_name)
                file_name_unzipped = file_name[:-3]
                command = f'gunzip -c {file_path} | NanoFilt \
                          --length {minimum_length} \
                          --maxlength {maximum_length} \
                          -q {qscore} | gzip > ./results/qc/{base}/{file_name}'
                subprocess.run(command, shell=True)
                
'''
Function to concatenate filtered sequences to one file per sample
'''
def concatenate(bases):
    for base in bases:
        command = f'cat ./results/qc/{base}/*.fastq.gz > ./results/qc/{base}/{base}_concatenated.fastq.gz'
        subprocess.run(command, shell=True)
        
        command = f'gunzip ./results/qc/{base}/{base}_concatenated.fastq.gz'
        subprocess.run(command, shell=True)
        
        command = f'sed -n "1~4s/^@/>/p;2~4p" ./results/qc/{base}/{base}_concatenated.fastq > ./results/qc/{base}/{base}_concatenated.fasta'
        subprocess.run(command, shell=True)

        #command = f'gzip ./results/qc/{base}/{base}_concatenated.fasta'
        #subprocess.run(command, shell=True)
        
'''
Function to count the number of sequences per sample after filtering
'''
def count_sequences_concat(base_name):
    counts = {}
    for base in base_name:
        counts[base] = 0  # Initialize the count for each base
        file_path = './results/qc/' + base + '/' + base + '_concatenated.fasta'
        with open(file_path, 'r') as file:
            for line in file:
                if line.startswith('>'):
                    counts[base] += 1
    return counts

'''
Function to convert sequence files in fasta format to csv
'''
def fasta2csv(base_name):
    for base in base_name:
        fasta = './results/qc/' + base + '/' + base + '_concatenated.fasta'
        output = './results/qc/' + base + '/' + base + '_concatenated.csv'

        out_lines = []
        temp_line = ''
        with open(fasta, 'r') as fp:
            for line in fp:
                if line.startswith('>'):
                    out_lines.append(temp_line)
                    temp_line = line.strip() + ','
                else:
                    temp_line += line.strip()
        out_lines.append(temp_line)

        with open(output, 'w') as fp_out:
            fp_out.write('id,sequence' + '\n'.join(out_lines))
            
'''
Wrapper function to run the ashure clustering algorithm
'''
def cluster(base):
    os.chdir(wdir)
    shutil.copy('./ashure.py', './results/qc/' + base)
    shutil.copy('./bilge_pype.py', './results/qc/' + base)
    os.chdir('./results/qc/' + base)
    script_path = "./ashure.py"
    input_file = base + "_concatenated.csv"
    output_file = base + "_clusters.csv"    
    
    command = [
        script_path,
        "clst",
        "-i", input_file,
        "-o", output_file,
        "-iter", config.get('ashure', 'niter'),
        "-r"
    ]   
    
    subprocess.run(command)
    os.chdir(wdir)

'''
Function to convert sequence files in csv format to fasta
'''
def csv2fasta(base_name):
    for base in base_name:
        csv_path = './results/qc/' + base + '/' + base + '_clusters.csv'
        output_path = './results/qc/' + base + '/' + base + '_clusters.fasta'

        if os.path.exists(csv_path):
            out_lines = []
            temp_line = ''
            with open(csv_path, 'r') as csv_file:
                for line in csv_file:
                    cols = line.split(",")
                    out_lines.append(temp_line)
                    temp_line = ">" + cols[0] + "\n" + cols[1] + "\n"

            out_lines.append(temp_line)

            with open(output_path, 'w') as csv_out:
                csv_out.write(''.join(out_lines)[13:])
                
'''
Wrapper function to run cutadapt
'''
def remove_primers(base_name):
    for base in base_name:
        file_path = './results/qc/' + base + '/' + base + '_clusters.fasta'
        out_path = './results/qc/' + base + '/' + base + '_clusters_cut.fasta'
        if os.path.exists(file_path):
            command = [
                'cutadapt',
                '-a', 'CAGCAGCCGCGGTAATTCC;max_error_rate=0.20',
                '-g', 'CCCGTGTTGAGTCAAATTAAGC;max_error_rate=0.20',
                '--revcomp',
                '-o', out_path,
                file_path
            ]
            subprocess.run(command)

'''
Wrapper function to run blastn
'''                        
def blast(base_name):
    for base in base_name:
        file_path = './results/qc/' + base + '/' + base + '_clusters_cut.fasta'
        db = config.get('BLAST', 'db')
        if os.path.exists(file_path):
            print("Running blastn on", base)
            output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'

            command = [
                "blastn",
                "-db", config.get('paths', 'path_to_blastdb'),
                "-query", file_path,
                "-task", "blastn",
                "-dust", "no",
                "-num_threads", str(config.get('BLAST', 'numthreads')),
                "-outfmt", "7 delim=, sseqid stitle qacc sacc evalue bitscore length pident",
                "-max_target_seqs", str(config.get('BLAST', 'mts')),
                "-perc_identity", str(config.get('BLAST', 'pct_ident')),
                "-out", output_csv
            ]

            subprocess.run(command)

'''
Function to handle the blastn output files and generate a concatenated table with the taxonomic annotations
'''
def make_output_file(base_name):
    db = config.get('BLAST', 'db')
    for base in base_name:
        input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'
        output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
        if os.path.exists(input_csv):
            with open(input_csv, 'r') as infile, open(output_csv, 'w') as outfile:
                for line in infile:
                    if not line.startswith('#'):
                        outfile.write(line)
                    
    for base in base_name:
        input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
        print(input_csv)
        output_csv = './results/qc/' + base + '/' + base + '_' + db + '_ASV.csv'
        if os.path.exists(input_csv) and os.path.getsize(input_csv) > 0:
            # load file
            df = pd.read_csv(input_csv, sep=',')

            # add column names
            df.columns=['accession', 'taxonomic_annotation', 'cluster', 'accession', 'evalue', 'bitscore', 'alignment_length', 'percentage_identity']

            # select only rows with alignment length >= 500 bp
            df2 = df[df['alignment_length'] >= 500]

            # arrange rows by match percentage
            df3 = df2.sort_values(by=['percentage_identity'], ascending=False)

            # keep only first row of each ASV
            df4 = df3.drop_duplicates(subset=['cluster'], keep='first', inplace=False, ignore_index=False)

            # add sample name information
            df4['#sample_name'] = base

            df4['taxonomy'] = df4['taxonomic_annotation'].replace('"', '')

            df5 = df4[['#sample_name', 'cluster', 'accession', 'evalue', 'bitscore', 'alignment_length', 'percentage_identity', 'taxonomic_annotation']]

            df5.to_csv(output_csv, sep=';', index=False, header=False)

    intermediate = '_' + db + '_eDNA.csv'
    final = db + '_eDNA.csv'
    if os.path.exists(intermediate):
        os.remove(intermediate)

    for base in base_name:
        file_path = './results/qc/' + base + '/' + base + '_' + db + '_ASV.csv'
        if os.path.exists(file_path):
            with open(file_path, "r") as input_file, open(intermediate, "a") as output_file:
                output_file.write(input_file.read())

    with open(intermediate, "r") as input_file, open(final, "w") as output_file:
        output_file.write("counts,cluster,accession,accession,evalue,bitscore,alignment_length,percentage_identity,taxonomic_annotation\n")
        for line in input_file:
            if not line.startswith("#"):
                output_file.write(line.replace(";", ",").replace("|", ","))

## Actual execution of the workflow

In [52]:

# Load paths to directories where fastq files are located
fastq_dir = fastq_directory_paths()
print(fastq_dir)

# Extract sample names from data
base_name = basename(fastq_dir)
print(base_name)

# Count number of raw sequences per sample
print(accumulate_counts(fastq_dir))

# Filter sequences using NanoFilt
filtering(fastq_dir, config.get('NanoFilt', 'minlength'), config.get('NanoFilt', 'maxlength'), config.get('NanoFilt', 'qscore'), base_name)

# Concatenate filtered sequences to one file per sample
concatenate(base_name)
        
# Count number of sequences per sample after filtering
print(count_sequences_concat(base_name))

# Convert fasta files to csv
fasta2csv(base_name)

# Make sure you save the original working directory before using moving around directories
wdir = os.getcwd()
print(wdir)

# Run the actual clustering
if __name__ == '__main__':
    num_processes = 8  # Number of available CPU cores 
    with multiprocessing.Pool(processes=num_processes) as pool:
        pool.map(cluster, base_name)

# Move back to original working directory after clustering
os.chdir(wdir)

# Convert csv files to fasta
csv2fasta(base_name)

# Remove primers using a wrapper function for cutadapt
remove_primers(base_name)

# Taxonomic annotation using blastn
blast(base_name)

# Handle the blastn output files and generate a concatenated table with the taxonomic annotations
make_output_file(base_name)

['../data/barcode01', '../data/barcode02', '../data/barcode03']
['barcode01', 'barcode02', 'barcode03']
{'../data/barcode01': 28513, '../data/barcode02': 28554, '../data/barcode03': 28469}
{'barcode01': 25040, 'barcode02': 26721, 'barcode03': 20929}
/home/pascal/Documents/git_projects/grifo/src_0.4
pid[178468] 2023-06-24 23:44:04.481 INFO: check_toolchain: /home/pascal/anaconda3/envs/GEANS/bin/minimap2 found
pid[178469] 2023-06-24 23:44:04.481 INFO: check_toolchain: /home/pascal/anaconda3/envs/GEANS/bin/minimap2 found
pid[178468] 2023-06-24 23:44:04.481 INFO: check_toolchain: /usr/bin/bwa found
pid[178469] 2023-06-24 23:44:04.481 INFO: check_toolchain: /usr/bin/bwa found
pid[178468] 2023-06-24 23:44:04.481 INFO: check_toolchain: /usr/bin/bowtie2 found
pid[178469] 2023-06-24 23:44:04.481 INFO: check_toolchain: /usr/bin/bowtie2 found
pid[178468] 2023-06-24 23:44:04.481 INFO: check_toolchain: /home/pascal/anaconda3/envs/GEANS/bin/spoa found
pid[178469] 2023-06-24 23:44:04.481 INFO: check_

pid[178467] 2023-06-24 23:44:04.731 INFO: cluster_sample: qlen=26721 progress=0/5
pid[178467] 2023-06-24 23:44:04.734 INFO: Making directory ./clusters/


[M::mm_idx_gen::0.000*9.63] collected minimizers
[M::mm_idx_gen::0.001*4.92] sorted minimizers
[M::main::0.001*4.86] loaded/built the index for 0 target sequence(s)
[M::mm_mapopt_update::0.001*4.74] mid_occ = 0
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 0
[M::mm_idx_stat::0.001*4.65] distinct minimizers: 0 (-nan% are singletons); average occurrences: -nan; average spacing: -nan
[M::worker_pipeline::0.085*2.43] mapped 20929 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -c -k15 -w10 -p 0.9 -D -o ./clusters/results.paf ./clusters/database.fq ./clusters/read1.fq
[M::main] Real time: 0.086 sec; CPU: 0.207 sec; Peak RSS: 0.116 GB
[M::mm_idx_gen::0.000*11.37] collected minimizers
[M::mm_idx_gen::0.001*5.56] sorted minimizers
[M::main::0.001*5.51] loaded/built the index for 0 target sequence(s)
[M::mm_mapopt_update::0.001*5.38] mid_occ = 0
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 0
[M::mm_idx_stat::0.001*5.29] distinct minimizers: 0 (-nan% are

pid[178468] 2023-06-24 23:44:05.652 INFO: cluster_sample: qlen=20929 progress=1/5
pid[178469] 2023-06-24 23:44:05.825 INFO: cluster_sample: qlen=25040 progress=1/5


[M::worker_pipeline::0.108*2.35] mapped 25040 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -c -k15 -w10 -p 0.9 -D -o ./clusters/results.paf ./clusters/database.fq ./clusters/read1.fq
[M::main] Real time: 0.108 sec; CPU: 0.254 sec; Peak RSS: 0.120 GB
[M::mm_idx_gen::0.001*9.81] collected minimizers
[M::mm_idx_gen::0.001*5.71] sorted minimizers
[M::main::0.001*5.70] loaded/built the index for 0 target sequence(s)
[M::mm_mapopt_update::0.001*5.53] mid_occ = 1725152489
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 0
[M::mm_idx_stat::0.001*5.42] distinct minimizers: 0 (-nan% are singletons); average occurrences: -nan; average spacing: -nan
[M::worker_pipeline::0.119*2.49] mapped 26721 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -c -k15 -w10 -p 0.9 -D -o ./clusters/results.paf ./clusters/database.fq ./clusters/read1.fq
[M::main] Real time: 0.119 sec; CPU: 0.297 sec; Peak RSS: 0.122 GB


...

[M::worker_pipeline::1.727*2.94] mapped 2000 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -c -k15 -w10 -p 0.9 -D --dual=no --for-only -o ./clusters/results.paf ./clusters/database.fq ./clusters/read1.fq
[M::main] Real time: 1.731 sec; CPU: 5.078 sec; Peak RSS: 0.167 GB
[M::worker_pipeline::2.006*2.91] mapped 20929 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -c -k15 -w10 -p 0.9 -D --for-only -o ./clusters/results.paf ./clusters/database.fq ./clusters/read1.fq
[M::main] Real time: 2.008 sec; CPU: 5.845 sec; Peak RSS: 0.143 GB


pid[178467] 2023-06-24 23:45:56.655 INFO: clust_OPTICS: clusters=1 outliers=9 delta=7
pid[178467] 2023-06-24 23:45:56.655 INFO: clust_OPTICS: iter=8 using min_samples=7


/home/pascal/anaconda3/envs/GEANS/lib/python3.8/site-packages/sklearn/cluster/_optics.py:570: UserWarning: All reachability values are inf. Set a larger max_eps or all data will be considered outliers.
  warnings.warn(
./ashure.py:746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_o['outlier'] = True
./ashure.py:612: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_c = df_c.append(cout[['id','sequence','split']])
/home/pascal/anaconda3/envs/GEANS/lib/python3.8/site-packages/sklearn/cluster/_optics.py:570: UserWarning: All reachability values are inf. Set a larger max_eps or all data will be considered outliers.
  warnings.warn(
./ashure.py:746: SettingWith

KeyboardInterrupt: 

e/pascal/anaconda3/envs/GEANS/lib/python3.8/site-packages/sklearn/cluster/_optics.py", line 595, in _set_reach_dist
    indices = nbrs.radius_neighbors(P, radius=max_eps, return_distance=False)[0]
  File "/home/pascal/anaconda3/envs/GEANS/lib/python3.8/site-packages/sklearn/neighbors/_base.py", line 1084, in radius_neighbors
    neigh_ind_list = sum(chunked_results, [])
  File "/home/pascal/anaconda3/envs/GEANS/lib/python3.8/site-packages/sklearn/metrics/pairwise.py", line 1717, in pairwise_distances_chunked
    D_chunk = pairwise_distances(X_chunk, Y, metric=metric, n_jobs=n_jobs, **kwds)
  File "/home/pascal/anaconda3/envs/GEANS/lib/python3.8/site-packages/sklearn/metrics/pairwise.py", line 1851, in pairwise_distances
    X, _ = check_pairwise_arrays(
  File "/home/pascal/anaconda3/envs/GEANS/lib/python3.8/site-packages/sklearn/metrics/pairwise.py", line 164, in check_pairwise_arrays
    Y = check_array(
  File "/home/pascal/anaconda3/envs/GEANS/lib/python3.8/site-packages/sklearn/ut

## Taxonomic assignment using PR2

In [35]:
path_to_blastdb = "/home/pascal/Documents/GEANS/eDNA_18S/PR2_5.0.0/pr2_version_5.0.0_SSU_taxo_long.fasta"
numthreads = 8
mts = 10
pct_ident = 90
db = 'PR2'

for base in base_name:
    file_path = './results/qc/' + base + '/' + base + '_clusters_cut.fasta'
    if os.path.exists(file_path):
        print("Running blastn on", base)
        output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'

        command = [
            "blastn",
            "-db", path_to_blastdb,
            "-query", file_path,
            "-task", "blastn",
            "-dust", "no",
            "-num_threads", str(numthreads),
            "-outfmt", "7 delim=, sseqid stitle qacc sacc evalue bitscore length pident",
            "-max_target_seqs", str(mts),
            "-perc_identity", str(pct_ident),
            "-out", output_csv
        ]

        subprocess.run(command)

Running blastn on barcode01


Running blastn on barcode02


Running blastn on barcode03


Running blastn on barcode04


Running blastn on barcode05


Running blastn on barcode06


Running blastn on barcode07


Running blastn on barcode08
Running blastn on barcode09
Running blastn on barcode10


Running blastn on barcode11
Running blastn on barcode12


In [36]:
db = 'PR2'


for base in base_name:
    input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'
    output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
    if os.path.exists(input_csv):
        with open(input_csv, 'r') as infile, open(output_csv, 'w') as outfile:
            for line in infile:
                if not line.startswith('#'):
                    outfile.write(line)



In [37]:
db = 'PR2'

for base in base_name:
    input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn.csv'
    output_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
    if os.path.exists(input_csv):
        with open(input_csv, 'r') as infile, open(output_csv, 'w') as outfile:
            for line in infile:
                if not line.startswith('#'):
                    comma_count = line.count(',')
                    if comma_count == 10:
                        comma_indices = []
                        for i, char in enumerate(line):
                            if char == ',':
                                comma_indices.append(i)

                        if len(comma_indices) >= 6:
                            line = line[:comma_indices[0]] + line[comma_indices[0]+1:]
                            line = line[:comma_indices[2]-1] + line[comma_indices[2]:]
                            line = line[:comma_indices[5]-2] + line[comma_indices[5]-1:]
                    elif comma_count == 13:
                        comma_indices = []
                        for i, char in enumerate(line):
                            if char == ',':
                                comma_indices.append(i)

                        indices_to_remove = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # Create a list to track the indices to remove

                        if len(comma_indices) >= 13:
                            indices_to_remove[0] = 1  # Mark the first comma to remove
                            indices_to_remove[1] = 1  # Mark the second comma to remove
                            indices_to_remove[3] = 1  # Mark the fourth comma to remove
                            indices_to_remove[4] = 1  # Mark the fifth comma to remove
                            indices_to_remove[7] = 1  # Mark the eighth comma to remove
                            indices_to_remove[8] = 1  # Mark the ninth comma to remove

                        updated_line = ""
                        for i, char in enumerate(line):
                            if char == ',' and indices_to_remove[comma_indices.index(i)] == 1:
                                continue
                            updated_line += char

                        line = updated_line
                    outfile.write(line)

In [38]:

db = 'PR2'




for base in base_name:
    input_csv = './results/qc/' + base + '/' + base + '_' + db + '_blastn2.csv'
    print(input_csv)
    output_csv = './results/qc/' + base + '/' + base + '_' + db + '_ASV.csv'
    if os.path.exists(input_csv) and os.path.getsize(input_csv) > 0:
        # load file
        df = pd.read_csv(input_csv, sep=',')
    
        # add column names
        df.columns=['accession', 'taxonomic_annotation', 'cluster', 'accession', 'evalue', 'bitscore', 'alignment_length', 'percentage_identity']

        # select only rows with alignment length >= 500 bp
        df2 = df[df['alignment_length'] >= 500]

        # arrange rows by match percentage
        df3 = df2.sort_values(by=['percentage_identity'], ascending=False)

        # keep only first row of each ASV
        df4 = df3.drop_duplicates(subset=['cluster'], keep='first', inplace=False, ignore_index=False)

        # add sample name information
        df4['#sample_name'] = base

        df4['taxonomy'] = df4['taxonomic_annotation'].replace('"', '')

        df5 = df4[['#sample_name', 'cluster', 'accession', 'evalue', 'bitscore', 'alignment_length', 'percentage_identity', 'taxonomic_annotation']]


        df5.to_csv(output_csv, sep=';', index=False, header=False)


./results/qc/barcode01/barcode01_PR2_blastn2.csv
./results/qc/barcode02/barcode02_PR2_blastn2.csv
./results/qc/barcode03/barcode03_PR2_blastn2.csv
./results/qc/barcode04/barcode04_PR2_blastn2.csv
./results/qc/barcode05/barcode05_PR2_blastn2.csv
./results/qc/barcode06/barcode06_PR2_blastn2.csv
./results/qc/barcode07/barcode07_PR2_blastn2.csv
./results/qc/barcode08/barcode08_PR2_blastn2.csv
./results/qc/barcode09/barcode09_PR2_blastn2.csv
./results/qc/barcode10/barcode10_PR2_blastn2.csv
./results/qc/barcode11/barcode11_PR2_blastn2.csv
./results/qc/barcode12/barcode12_PR2_blastn2.csv


/tmp/ipykernel_1122442/2543769090.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['#sample_name'] = base
/tmp/ipykernel_1122442/2543769090.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['taxonomy'] = df4['taxonomic_annotation'].replace('"', '')
/tmp/ipykernel_1122442/2543769090.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [39]:
if os.path.exists("_PR2_eDNA.csv"):
    os.remove("_PR2_eDNA.csv")

for base in base_name:
    file_path = './results/qc/' + base + '/' + base + '_' + db + '_ASV.csv'
    if os.path.exists(file_path):
        with open(file_path, "r") as input_file, open("_PR2_eDNA.csv", "a") as output_file:
            output_file.write(input_file.read())

with open("_PR2_eDNA.csv", "r") as input_file, open("PR2_eDNA.csv", "w") as output_file:
    output_file.write("counts,cluster,accession,accession,evalue,bitscore,alignment_length,percentage_identity,taxonomic_annotation\n")
    for line in input_file:
        if not line.startswith("#"):
            output_file.write(line.replace(";", ",").replace("|", ","))

#shutil.copy("PR2_eDNA.csv", os.path.join(current_dir, "PR2_eDNA.csv"))